**Import needed packages/modules**

In [ ]:
# Cell 1
import matplotlib.pyplot as plt
import numpy as np
from numba import float64, int64, vectorize

**Declare a numba accelerated function that computes the Halton QRNG**
1. The parameter $n$ is an integer of any size
2. The parameter $p$ is a prime number

In [ ]:
# Cell 2
@vectorize([float64(int64, int64)], nopython=True)
def halton(n, p):
    h, f = 0, 1
    while n > 0:
        f = f / p
        h += (n % p) * f
        n = int(n / p)
    return h

**Set the total number of dots (samples) to take**

In [ ]:
# Cell 3
total_dots = 125_000

**Create `total_dots` samples of 3D Cartesian points $(x,y,z)$ using the Halton sequence**
1. The Halton QRNG returns a random float [0,1)
2. Subtract that float from 1, so the interval flips to (0,1] ensuring any points on the perimeter will now contribute to the volume
3. Scale the result so it now falls in the interval [-1, 1]

In [ ]:
# Cell 4
x = (1 - halton(np.arange(total_dots), 2)) * 2 - 1
y = (1 - halton(np.arange(total_dots), 3)) * 2 - 1
z =

**Create an array $d$ that contains the distance from the origin $(0,0)$ for every point $(x,y,z)$**\
Leverage the fact the exponentiation and addition operators are "vector aware"

In [ ]:
# Cell 5
d = x**2 + y**2 +

**Create arrays of $(x,y)$ coordinates that are "on or inside" vs. "outside" the sphere using the Pythagorean distance $d$**\
Leverage the ability to `filter` numpy arrays using a conditional expression

In [ ]:
# Cell 6

# On the surface (or inside) the sphere
x_in = x[d <= 1.0]
y_in = y[d <= 1.0]
z_in =

# Outside the sphere
x_out = x[d > 1.0]
y_out = y[d > 1.0]
z_out =

**Calculate the absolute percent error in the area estimation**
1. The actual/expected volume of a unit sphere is exactly $\frac{4}{3}\pi$
2. The observed/estimated volume using the Monte Carlo formulation $\large=8\times\frac{dots_{\ inside}}{dots_{\ total}}$


In [ ]:
# Cell 7
act =
est =  * np.count_nonzero(d <= 1.0) / total_dots
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")

**Display the scatter plot of the Monte Carlo estimation**

In [ ]:
# Cell 8
ax = plt.axes(projection="3d")
ax.view_init(azim=-72, elev=48)
ax.scatter(x_in, y_in, z_in, color="red", marker=".", s=0.1)
ax.scatter(x_out, y_out, z_out, color="blue", marker=".", s=0.1)
plt.gcf().set_size_inches(10, 10)
plt.gca().set_aspect("equal")
plt.show()